### Постановка задачи
Загрузить данные и посчитать модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей: 
- в первой модели весь оптимальный набор метеорологических данных
- во второй модели содержатся все дни недели и праздники
- в третьей модели описаны недели года
- в четвертой - месяцы. 

Финальное значение показателя рассчитайте как взвешенное арифметическое показателей всех моделей, взяв веса для первой и второй модели как 3/8, а для третьей и четвертой - как 1/8.

Загрузить данные решения, посчитать значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузите результат в виде CSV-файла (submission.csv).

Сколько строк в итоговом файле имеют ненулевое значение для показателя meter_reading ?

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/test.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz

Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [ ]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar     # импортируем праздники США из временных данных pandas
import numpy as np
from scipy.interpolate import interp1d      #  нахождение значений при помощи интерполяции одномерной функции
from sklearn.linear_model import LinearRegression    # построение модели линейной регрессии через полученные значения

In [ ]:
pd.set_option("display.max_rows", 25)             # настройка отображения результатов работы с данными в pandas
pd.set_option("display.max_columns", 25)
pd.set_option("display.max_colwidth", 25)
pd.set_option("display.width", 1000)

### Загрузка данных для расчета, оптимизация памяти

In [ ]:
def reduce_mem_usage (df):                          # функция оптимизации(сокращения) занимаемой на диске памяти загружаемых файлов
    start_mem = df.memory_usage().sum() / 1024**2      # начальный объем файла на диске
    for col in df.columns:     # переопределяем тип данных для отдельных серий по загруженным колонкам данных
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":    # оптимизируем память для вещественных типов данных 
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":    # оптимизируем память для целочисленных типов данных
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":      # оптимизируем тип данных для хранения даты и времени
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":    # остальные данные относим к категориальному формату данных
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2    # объем файла на диске после оптимизации
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

### Загрузка данных для модели по 50 зданиям

In [ ]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
weather = weather[weather["site_id"] == 0]
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
energy = energy[energy["building_id"]<50]         # выбираем первые 50 зданий из загруженных данных
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")   #  объединяем данные по зданиям и энергопотреблению в один dataframe
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432456 entries, 0 to 432455
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   building_id    432456 non-null  int64  
 1   meter          432456 non-null  int64  
 2   timestamp      432456 non-null  object 
 3   meter_reading  432456 non-null  float64
 4   site_id        432456 non-null  int64  
 5   primary_use    432456 non-null  object 
 6   square_feet    432456 non-null  int64  
 7   year_built     432456 non-null  float64
 8   floor_count    0 non-null       float64
dtypes: float64(3), int64(4), object(2)
memory usage: 33.0+ MB
None


### Интерполяция и обогащение данных по погоде

In [ ]:
def weather_interpolate_diff (weather):     # функция заполнения отсутствующих данных по погоде усредненными по смежным значениям
    interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed",
                       "sea_level_pressure"]    # колонки с погодными данными для интерполяции
    for col in interpolate_columns:
        weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic')
    weather["air_temperature_diff1"] = weather["air_temperature"].diff()   # вычисление разницы между смежными значениями
    weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
    weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
    weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]
    return weather

In [ ]:
weather = weather_interpolate_diff(weather)

### Объединение данных с погодными

In [ ]:
energy = energy.set_index(["timestamp", "site_id"])      # группируем данные по индексным колонкам 
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",   
                  left_index=True, right_index=True)      # объединяем данные в общий dataframe
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "year_built",
                              "square_feet", "floor_count"], axis=1)   # удаление неиспользуемых колонок данных
del weather
energy = reduce_mem_usage(energy)    # уменьшаем размер используемой памяти для объединенного dataframe
print (energy.info())

Потребление памяти меньше на 33.41 Мб (минус 72.3 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432456 entries, 0 to 432455
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              432456 non-null  datetime64[ns]
 1   site_id                432456 non-null  int8          
 2   building_id            432456 non-null  int8          
 3   meter_reading          432456 non-null  float16       
 4   primary_use            432456 non-null  category      
 5   air_temperature        432456 non-null  float16       
 6   cloud_coverage         432456 non-null  float16       
 7   dew_temperature        432456 non-null  float16       
 8   precip_depth_1_hr      432408 non-null  float16       
 9   sea_level_pressure     432456 non-null  float16       
 10  wind_direction         420144 non-null  float16       
 11  wind_speed             432456 non-null  float16    

### Обогащение данных по датам и логарифмирование

In [ ]:
def add_dates_log (df):            # функция приведения временных данных к целочисленному типу
    df["hour"] = df["timestamp"].dt.hour.astype("int8")
    df["weekday"] = df["timestamp"].dt.weekday.astype("int8")  # для колонок загруженного dataframe
    df["week"] = df["timestamp"].dt.week.astype("int8") 
    df["month"] = df["timestamp"].dt.month.astype("int8")
    df["date"] = pd.to_datetime(df["timestamp"].dt.date)
    for weekday in range(0,7):    # для каждой строки данных вводим параметры(флаги)-принадлежность к определенной неделе в году(для корректировки предсказания по сезонам)
        df['is_wday' + str(weekday)] = df['weekday'].isin([weekday]).astype("int8")  # для дней недели
    for week in range(1,54):
        df['is_w' + str(week)] = df['week'].isin([week]).astype("int8")   # для недель в году
    for month in range(1,13):
        df['is_m' + str(month)] = df['month'].isin([month]).astype("int8")   # для месяцев в году
    dates_range = pd.date_range(start='2015-12-31', end='2018-06-01')
    us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
    df['is_holiday'] = df['date'].isin(us_holidays).astype("int8")    # для праздничных дней в США
    if "meter_reading" in df.columns:
        df["meter_reading_log"] = np.log(df["meter_reading"] + 1)   # логарифмируем показания счетчика для проверки перехода от модели суммы показателей к модели произведения
    return df

In [ ]:
energy = add_dates_log(energy)     # оптимизация временных данных и логарифмирование ключевого показател
print(energy.head())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


   timestamp  site_id  building_id  meter_reading primary_use  air_temperature  cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  wind_speed  ...  is_m3  is_m4  is_m5  is_m6  is_m7  is_m8 is_m9  is_m10  is_m11  is_m12  is_holiday  meter_reading_log
0 2016-01-01        0            0            0.0   Education             25.0             6.0             20.0                NaN              1019.5             0.0         0.0  ...      0      0      0      0      0      0     0       0       0       0           1                0.0
1 2016-01-01        0            1            0.0   Education             25.0             6.0             20.0                NaN              1019.5             0.0         0.0  ...      0      0      0      0      0      0     0       0       0       0           1                0.0
2 2016-01-01        0            2            0.0   Education             25.0             6.0             20.0                NaN         

### Расчет моделей линейной регрессии

In [ ]:
hours = range(0, 24)      # расчет моделей по часам
buildings = range(0, energy["building_id"].max() + 1)

In [ ]:
def calculate_model_coeffs (columns):    # функция расчета коэф-ов уравнения линейной регрессии у = ах + в; (а, в -> coef_, intercept_)
  energy_train_lr = pd.DataFrame(energy, columns=columns)
  coeffs = [[]]*len(buildings)
  for building in buildings:      # для каждого здания
    coeffs[building] = [[]]*len(hours)
    energy_train_b = energy_train_lr[energy_train_lr["building_id"]==building]
    for hour in hours:            # перебираем почасовое энергопотребление
      energy_train_bh = energy_train_b[energy_train_b["hour"]==hour]
      y = energy_train_bh["meter_reading_log"]
      x = energy_train_bh.drop(labels=["meter_reading_log",
                    "hour", "building_id"], axis=1)
      model = LinearRegression(fit_intercept=False).fit(x, y)
      coeffs[building][hour] = model.coef_
      coeffs[building][hour] = np.append(coeffs[building][hour], model.intercept_)
  return coeffs

lr_columns_weather = ["meter_reading_log", "hour", "building_id",
                      "air_temperature", "dew_temperature",
                      "sea_level_pressure", "wind_speed", "cloud_coverage",
                      "air_temperature_diff1", "air_temperature_diff2"]
energy_lr_w = calculate_model_coeffs(lr_columns_weather)    # модель LR для энергопотребления по метеорологич.данным
print (energy_lr_w[0])

[array([ 0.05956934,  0.22113499,  0.0009459 , -0.19715199, -0.2873579 ,
        0.40280262,  0.14212959,  0.        ]), array([ 0.08644635,  0.17929874,  0.00108309, -0.23158213, -0.22974287,
        0.78134131, -0.56720734,  0.        ]), array([ 5.38871624e-02,  2.21623257e-01,  2.65777111e-04, -1.53262198e-01,
       -1.97125703e-01,  7.48542249e-01, -3.42919111e-01,  0.00000000e+00]), array([ 1.18710123e-01,  1.56611592e-01, -1.95607543e-04, -2.13227361e-01,
       -1.64893553e-01,  6.96503043e-01, -4.38024551e-01,  0.00000000e+00]), array([ 0.147507  ,  0.13394338, -0.0011562 , -0.06129656, -0.11472981,
        0.51194078, -0.23469272,  0.        ]), array([ 0.29722789, -0.01745704, -0.00124428, -0.06564673, -0.1266879 ,
        1.01383615, -0.42186883,  0.        ]), array([ 0.34338889, -0.05661482, -0.00150706, -0.066377  , -0.1270553 ,
        0.66892225, -0.65648937,  0.        ]), array([ 0.35005933, -0.04034004, -0.00206696, -0.06744336, -0.18052113,
       -0.36294156,  0.

In [ ]:
lr_columns_days = ["meter_reading_log", "hour", "building_id",
                   "is_holiday"]
for wday in range(0,7):
  lr_columns_days.append("is_wday" + str(wday))
energy_lr_d = calculate_model_coeffs(lr_columns_days)     # модель LR для энергопотребления по дням
print (energy_lr_d[0])

[array([0.46404106, 3.28496382, 3.36222957, 3.3816857 , 3.3443664 ,
       3.27391001, 3.39121462, 3.34728065, 0.        ]), array([0.48017462, 3.28346808, 3.35539363, 3.37800481, 3.35104232,
       3.27521747, 3.39460495, 3.3534405 , 0.        ]), array([0.46185674, 3.28668515, 3.35389123, 3.38176082, 3.35642763,
       3.27863571, 3.39770047, 3.34743089, 0.        ]), array([0.44014085, 3.29268837, 3.357497  , 3.37988281, 3.35429116,
       3.27938148, 3.40197524, 3.35554387, 0.        ]), array([0.42982065, 3.29820924, 3.3500601 , 3.38123498, 3.35306234,
       3.28102387, 3.3957842 , 3.35261418, 0.        ]), array([0.42599252, 3.29692168, 3.35351563, 3.37169471, 3.35614077,
       3.27969427, 3.393352  , 3.42608173, 0.        ]), array([0.44164408, 3.29796979, 3.34044471, 3.36816406, 3.35178329,
       3.27416555, 3.39401533, 3.35441707, 0.        ]), array([0.4679073 , 3.30232197, 3.36095252, 3.37973257, 3.36690323,
       3.27825998, 3.39224646, 3.35366587, 0.        ]), array([

In [ ]:
lr_columns_weeks = ["meter_reading_log", "hour", "building_id"]
for week in range(1,54):
  lr_columns_weeks.append("is_w" + str(week))
energy_lr_ww = calculate_model_coeffs(lr_columns_weeks)     # модель LR для энергопотребления по неделям в году
print (energy_lr_ww[0])

[array([-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  1.57924107e+00,
        5.29296875e+00,  5.34988839e+00,  5.54185268e+00,  5.63169643e+00,
        5.51004464e+00,  5.50055804e+00,  5.49218750e+00,  5.68303571e+00,
        5.67578125e+00,  5.65066964e+00,  5.64118304e+00,  5.67354911e+00,
        5.74162946e+00,  5.54687500e+00,  5.50892857e+00,  5.52232143e+00,
        5.49330357e+00,  5.47712054e+00,  5.52120536e+00,  5.51450893e+00,
        5.50111607e+00,  5.50223214e+00,  5.36607143e+00,  5.34988839e+00,
        5.27120536e+00,  5.23046875e+00,  5.41294643e+00,  5.25781250e+00,
        4.87667411e+00,  5.05580357e+00,  5.31194196e+00,  5.42773438e+00,
       -6.15348060e-15, 

In [ ]:
lr_columns_monthes = ["meter_reading_log", "hour", "building_id"]
for month in range(1,13):
  lr_columns_monthes.append("is_m" + str(month))          # модель LR для энергопотребления по месяцам в году
energy_lr_m = calculate_model_coeffs(lr_columns_monthes)    
print (energy_lr_m[0])

[array([0.        , 0.        , 0.        , 0.        , 1.8859627 ,
       5.52747396, 5.61504536, 5.63634073, 5.50364583, 5.47013609,
       5.31289062, 5.16796875, 0.        ]), array([0.        , 0.        , 0.        , 0.        , 1.89024698,
       5.53242187, 5.61983367, 5.63961694, 5.50859375, 5.46610383,
       5.30898438, 5.16847278, 0.        ]), array([0.        , 0.        , 0.        , 0.        , 1.890625  ,
       5.53958333, 5.6155494 , 5.64642137, 5.51419271, 5.46396169,
       5.30351563, 5.17401714, 0.        ]), array([0.        , 0.        , 0.        , 0.        , 1.89264113,
       5.54205729, 5.60370464, 5.64629536, 5.51614583, 5.46950605,
       5.31236979, 5.19014617, 0.        ]), array([0.        , 0.        , 0.        , 0.        , 1.89352319,
       5.54583333, 5.59122984, 5.63293851, 5.52265625, 5.47101815,
       5.31549479, 5.18157762, 0.        ]), array([0.11781754, 0.        , 0.        , 0.        , 1.89705141,
       5.55026042, 5.59475806, 5.6097

### Удаление обучающих данных

In [ ]:
del energy

### Подготовка данных для расчетов

In [ ]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz")
weather = weather[weather["site_id"] == 0]
results = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/test.csv.gz")
results = results[(results["building_id"] < 50) & (results["meter"] == 0)]
results = pd.merge(left=results, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
del buildings
print (results.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 876000 entries, 0 to 875999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   row_id       876000 non-null  int64  
 1   building_id  876000 non-null  int64  
 2   meter        876000 non-null  int64  
 3   timestamp    876000 non-null  object 
 4   site_id      876000 non-null  int64  
 5   primary_use  876000 non-null  object 
 6   square_feet  876000 non-null  int64  
 7   year_built   876000 non-null  float64
 8   floor_count  0 non-null       float64
dtypes: float64(2), int64(5), object(2)
memory usage: 66.8+ MB
None


### Интерполяция значений и обогащение погодных данных: только для 1 города

In [ ]:
weather = weather_interpolate_diff(weather)

### Объединение данных по погоде

In [ ]:
results = results.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
results = pd.merge(left=results, right=weather, how="left",
                  left_index=True, right_index=True)
results.reset_index(inplace=True)
results = results.drop(columns=["meter", "site_id", "year_built",
                              "square_feet", "floor_count"], axis=1)
del weather
results = reduce_mem_usage(results)    # уменьшаем размер используемой памяти для объединенного dataframe
print (results.info())

Потребление памяти меньше на 60.15 Мб (минус 69.2 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              876000 non-null  datetime64[ns]
 1   row_id                 876000 non-null  int32         
 2   building_id            876000 non-null  int8          
 3   primary_use            876000 non-null  category      
 4   air_temperature        876000 non-null  float16       
 5   cloud_coverage         876000 non-null  float16       
 6   dew_temperature        876000 non-null  float16       
 7   precip_depth_1_hr      874500 non-null  float16       
 8   sea_level_pressure     876000 non-null  float16       
 9   wind_direction         852150 non-null  float16       
 10  wind_speed             876000 non-null  float16       
 11  air_temperature_diff1  876000 non-null  float16    

### Обогащение данных по дате

In [ ]:
results = add_dates_log(results)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


### Расчет финальных показателей, только энергопотребление, только 50 первых зданий
Потребуется ~6 Гб оперативной памяти. Если память компьютера ограничена, имеет смысл провести расчеты по частям: например, по 10 зданий

In [ ]:
def calculate_model (x, model, columns):
    return (np.sum([x[col] * model[i] for i,col in enumerate(columns[3:])])
            + model[len(columns)-3])

def calculate_ensemble (x):   # рассчет ансамбля моделей LR по энергопотреблению за просчитанные временные периоды
    lr = -1
    lr_w = calculate_model(x, 
            energy_lr_w[x.building_id][x.hour], lr_columns_weather)
    lr_d = calculate_model(x, 
            energy_lr_d[x.building_id][x.hour], lr_columns_days)
    lr_ww = calculate_model(x, 
            energy_lr_ww[x.building_id][x.hour], lr_columns_weeks)
    lr_m = calculate_model(x, 
            energy_lr_m[x.building_id][x.hour], lr_columns_monthes)
    lr = np.exp((lr_w*3 + lr_d*3 + lr_ww + lr_m)/8)
    if lr < 0 or lr != lr or lr*lr == lr:
        lr = 0
    x["meter_reading"] = lr
    return x

results = results.apply(calculate_ensemble, axis=1, result_type="expand")   # применение функции расчета и формирование результата

### Усечение данных до требуемого формата: row_id, meter_reading - и выгрузка результата

results_ready = pd.DataFrame(results, columns=["row_id", "meter_reading"])
print (results_ready.info())
print (len(results_ready[results_ready["meter_reading"]>0]))

### Загрузка всех данных для заполнения их нулями

In [ ]:
results = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/test.csv.gz",
                     usecols=["row_id"])
results = pd.merge(left=results, right=results_ready, how="left",
                  left_on="row_id", right_on="row_id")
results.fillna(value=0, inplace=True)
print (results.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 2 columns):
 #   Column         Dtype  
---  ------         -----  
 0   row_id         int64  
 1   meter_reading  float64
dtypes: float64(1), int64(1)
memory usage: 954.4 MB
None


### Выгрузка результатов в CSV файл

In [ ]:
print(results.count())
print(results.head())
results.to_csv(r"D:\submission.csv")

row_id           41697600
meter_reading    41697600
dtype: int64
   row_id  meter_reading
0       0      19.431891
1       1      13.463644
2       2       3.461884
3       3      23.199064
4       4      53.867803


### Освобождение памяти

In [ ]:
del results
del results_ready